# Model evaluation

Kendra Wyant  
January 16, 2025

### Set Up Environment

In [ ]:
suppressPackageStartupMessages(library(tidyverse))
suppressPackageStartupMessages(source("https://github.com/jjcurtin/lab_support/blob/main/format_path.R?raw=true"))
suppressPackageStartupMessages(library(tidyposterior))

path_models_lag <- format_path(str_c("studydata/risk/models/lag"))
path_shared <- format_path("studydata/risk/data_processed/shared")
path_processed <- format_path("studydata/risk/data_processed/lag")

options(knitr.kable.NA = '')

In [ ]:
test_metrics_0 <- read_csv(here::here(path_models_lag, 
                                        "test_metrics_1week_0_v1_nested.csv"), 
                              col_types = cols()) |> 
  filter(.metric == "roc_auc") |> 
  select(outer_split_num, "lag0" = .estimate)

test_metrics_24 <- read_csv(here::here(path_models_lag, 
                                       "test_metrics_1week_24_v1_nested.csv"),
                             col_types = cols()) |> 
  filter(.metric == "roc_auc") |> 
  select(outer_split_num, "lag24" = .estimate)

test_metrics_72 <- read_csv(here::here(path_models_lag, 
                                        "test_metrics_1week_72_v1_nested.csv"),
                              col_types = cols()) |> 
  filter(.metric == "roc_auc") |> 
  select(outer_split_num, "lag72" = .estimate)

test_metrics_168 <- read_csv(here::here(path_models_lag, 
                                        "test_metrics_1week_168_v1_nested.csv"), 
                              col_types = cols()) |> 
  filter(.metric == "roc_auc") |> 
  select(outer_split_num, "lag168" = .estimate)

test_metrics_336 <- read_csv(here::here(path_models_lag, 
                                       "test_metrics_1week_336_v1_nested.csv"),
                             col_types = cols()) |> 
  filter(.metric == "roc_auc") |> 
  select(outer_split_num, "lag336" = .estimate)

test_metrics_all <- test_metrics_0 |> 
  left_join(test_metrics_24, by = c("outer_split_num")) |> 
  left_join(test_metrics_72, by = c("outer_split_num")) |>
  left_join(test_metrics_168, by = c("outer_split_num")) |>
  left_join(test_metrics_336, by = c("outer_split_num")) |> 
  mutate(fold_num = rep(1:10, 3),
         repeat_num = c(rep(1, 10), rep(2, 10), rep(3, 10))) |> 
  select(-outer_split_num) |> 
  glimpse()

Rows: 30
Columns: 7
$ lag0       <dbl> 0.8751586, 0.8951046, 0.8915280, 0.9109163, 0.9107734, 0.94…
$ lag24      <dbl> 0.8570908, 0.8958414, 0.8922776, 0.9160835, 0.8868194, 0.93…
$ lag72      <dbl> 0.8491782, 0.8787232, 0.8719031, 0.9072133, 0.8739759, 0.92…
$ lag168     <dbl> 0.8504410, 0.8865674, 0.8222166, 0.9222604, 0.8391237, 0.91…
$ lag336     <dbl> 0.8168572, 0.8723169, 0.7801488, 0.8706801, 0.8447883, 0.89…
$ fold_num   <int> 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 1, 2, 3, 4, 5, 6, 7, 8, 9, 1…
$ repeat_num <dbl> 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,…

#### Model evaluation

In [ ]:
#| output: false

# Repeated CV (id = repeat, id2 = fold within repeat)
# with a common variance:  statistic ~ model + (model | id2/id)
set.seed(101)
pp <- test_metrics_all |> 
  rename(id = fold_num,
         id2 = repeat_num) |> 
  perf_mod(formula = statistic ~ model + (1 | id2/id),
         transform = tidyposterior::logit_trans,  # for skewed & bounded AUC
         iter = 3000, chains = 4, adapt_delta = .99, # increased iteration from 2000 to fix divergence issues
         family = gaussian, 
)  


SAMPLING FOR MODEL 'continuous' NOW (CHAIN 1).
Chain 1: 
Chain 1: Gradient evaluation took 0.000139 seconds
Chain 1: 1000 transitions using 10 leapfrog steps per transition would take 1.39 seconds.
Chain 1: Adjust your expectations accordingly!
Chain 1: 
Chain 1: 
Chain 1: Iteration:    1 / 3000 [  0%]  (Warmup)
Chain 1: Iteration:  300 / 3000 [ 10%]  (Warmup)
Chain 1: Iteration:  600 / 3000 [ 20%]  (Warmup)
Chain 1: Iteration:  900 / 3000 [ 30%]  (Warmup)
Chain 1: Iteration: 1200 / 3000 [ 40%]  (Warmup)
Chain 1: Iteration: 1500 / 3000 [ 50%]  (Warmup)
Chain 1: Iteration: 1501 / 3000 [ 50%]  (Sampling)
Chain 1: Iteration: 1800 / 3000 [ 60%]  (Sampling)
Chain 1: Iteration: 2100 / 3000 [ 70%]  (Sampling)
Chain 1: Iteration: 2400 / 3000 [ 80%]  (Sampling)
Chain 1: Iteration: 2700 / 3000 [ 90%]  (Sampling)
Chain 1: Iteration: 3000 / 3000 [100%]  (Sampling)
Chain 1: 
Chain 1:  Elapsed Time: 4.759 seconds (Warm-up)
Chain 1:                4.373 seconds (Sampling)
Chain 1:                9.1

In [ ]:
pp_tidy <- pp |> 
  tidy(seed = 123) 

q = c(.025, .5, .975)
test_metrics_all_pp_perf <- pp_tidy |> 
  group_by(model) |> 
  summarize(pp_median = quantile(posterior, probs = q[2]),
            pp_lower = quantile(posterior, probs = q[1]), 
            pp_upper = quantile(posterior, probs = q[3])) |> 
  mutate(model = factor(model, levels = c("lag0", "lag24", "lag72", "lag168", "lag336"),
                        labels = c("0 lag", "24 lag", "72 lag", "168 lag", "336 lag"))) |> 
  arrange(model)

test_metrics_all_pp_perf |> 
  write_csv(here::here(path_models_lag, "test_metrics_all_pp_perf.csv"))

pp_tidy |> 
  write_csv(here::here(path_models_lag, "pp_tidy.csv"))

test_metrics_all_pp_perf

# A tibble: 5 × 4
  model   pp_median pp_lower pp_upper
  <fct>       <dbl>    <dbl>    <dbl>
1 0 lag       0.892    0.872    0.910
2 24 lag      0.886    0.865    0.905
3 72 lag      0.874    0.851    0.894
4 168 lag     0.869    0.846    0.891
5 336 lag     0.851    0.825    0.874

See if brms replicates

In [ ]:
# library(brms)
# 
# data <- test_metrics_all |> 
#   rename(id = fold_num,
#          id2 = repeat_num) |> 
#   pivot_longer(cols = starts_with("lag"), 
#                names_to = "model", 
#                values_to = "statistic")
# 
# pp_brm <-  brm(
#   formula = statistic ~ model + (1 | id2/id), # folds nested in repeats
#   data = data,
#   family = gaussian(link = "logit"), # normal distribution w/auroc bounded between 0 and 1
#   chains = 4,
#   control = list(adapt_delta = 0.99), 
#   iter = 3000,
#   seed = 101
# )
# 
# pp_tidy <- posterior_predict(pp_brm, type = "response") |>  # rows observation, columns observations
#   as_tibble(.name_repair = "unique") |> 
#   pivot_longer(cols = everything(), 
#                names_to = "draw", 
#                values_to = "posterior") |>
#   bind_cols(data$model)


# Calculate AUC
# roc_result <- roc(your_data$outcome, rowMeans(predictions))
# auc_value <- auc(roc_result)
# 
# print(auc_value)

### Model Comparisons

#### Baseline Contrasts

In [ ]:
ci_baseline <- pp |>
  contrast_models(list("lag0", "lag0", "lag0", "lag0"), 
                  list("lag24", "lag72", "lag168", "lag336")) |> 
  summary(size = 0) |> 
  mutate(contrast = factor(contrast, 
                           levels = c("lag0 vs lag24", "lag0 vs lag72", "lag0 vs lag168", 
                                      "lag0 vs lag336"),
                           labels = c("0 vs. 24", "0 vs. 72", 
                                      "0 vs. 168", "0 vs. 336")))

ci_median_baseline <- pp |> 
  contrast_models(list("lag0", "lag0", "lag0", "lag0"), 
                  list("lag24", "lag72", "lag168", "lag336")) |>  
  group_by(contrast) |> 
  summarize(median = quantile(difference, .5)) |> 
  mutate(contrast = factor(contrast, 
                           levels = c("lag0 vs. lag24", "lag0 vs. lag72", "lag0 vs. lag168", 
                                      "lag0 vs. lag336"),
                           labels = c("0 vs. 24", "0 vs. 72", 
                                      "0 vs. 168", "0 vs. 336")))


ci_baseline <- ci_baseline |> 
  left_join(ci_median_baseline, by = c("contrast")) 

ci_baseline |> 
  write_csv(here::here(path_models_lag, "ci_baseline.csv"))

ci_baseline

# A tibble: 4 × 10
  contrast  probability    mean    lower  upper  size pract_neg pract_equiv
  <fct>           <dbl>   <dbl>    <dbl>  <dbl> <dbl>     <dbl>       <dbl>
1 0 vs. 168       1     0.0226  0.0160   0.0294     0        NA          NA
2 0 vs. 24        0.956 0.00599 0.000236 0.0119     0        NA          NA
3 0 vs. 336       1     0.0411  0.0332   0.0495     0        NA          NA
4 0 vs. 72        1     0.0182  0.0119   0.0248     0        NA          NA
# ℹ 2 more variables: pract_pos <dbl>, median <dbl>

#### Adjacent Contrasts

In [ ]:
ci_lag <- pp |>
  contrast_models(list("lag24", "lag72", "lag168"), 
                  list("lag72", "lag168", "lag336")) |> 
  summary(size = 0) |> 
  mutate(contrast = factor(contrast, 
                           levels = c("lag24 vs lag72", "lag72 vs lag168", 
                                      "lag168 vs lag336"),
                           labels = c("24 vs. 72", "72 vs. 168", "168 vs. 336")))

ci_median_lag <- pp |> 
  contrast_models(list("lag24", "lag72", "lag168"), 
                  list("lag72", "lag168", "lag336")) |>  
  group_by(contrast) |> 
  summarize(median = quantile(difference, .5)) |> 
  mutate(contrast = factor(contrast, 
                           levels = c("lag24 vs. lag72", "lag72 vs. lag168", 
                                      "lag168 vs. lag336"),
                           labels = c("24 vs. 72", "72 vs. 168", "168 vs. 336")))

ci_lag <- ci_lag |> 
  left_join(ci_median_lag, by = c("contrast")) |> 
  arrange(contrast)

ci_lag |> 
  write_csv(here::here(path_models_lag, "ci_lag.csv"))

ci_lag

# A tibble: 3 × 10
  contrast    probability    mean    lower  upper  size pract_neg pract_equiv
  <fct>             <dbl>   <dbl>    <dbl>  <dbl> <dbl>     <dbl>       <dbl>
1 24 vs. 72         0.998 0.0122   0.00594 0.0188     0        NA          NA
2 72 vs. 168        0.862 0.00443 -0.00221 0.0112     0        NA          NA
3 168 vs. 336       1     0.0185   0.0113  0.0259     0        NA          NA
# ℹ 2 more variables: pract_pos <dbl>, median <dbl>